In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
import nibabel as nib
import os
import tensorflow as tf
from tensorflow.keras.utils import to_categorical # type: ignore # type: ignore
from skimage.transform import resize


In [ ]:
print(pd.__version__)
print(np.__version__)
print(tf.__version__)
print(torch.__version__)
print("GPU is", "available" if torch.cuda.is_available() else "NOT AVAILABLE")

In [ ]:
VOLUME_DIR = "Dataset/volume_pt1" 
SEGMENTATION_DIR = "Dataset/volume_pt1/segementation"
IMG_SIZE = (128, 128) 

def load_nifti(file_path):
    """Load a NIfTI file and return the NumPy array"""
    print(f"Loading file: {file_path}") 
    nifti_img = nib.load(file_path) 
    return nifti_img.get_fdata()

def preprocess_data(volume_path, segmentation_path):
    """Load and preprocess volume and segmentation data"""
    volume_data = load_nifti(volume_path)
    segmentation_data = load_nifti(segmentation_path).astype(int)

    volume_data = (volume_data - np.min(volume_data)) / (np.max(volume_data) - np.min(volume_data))

    volume_resized = np.array([resize(slice, IMG_SIZE, mode='constant', preserve_range=True) for slice in volume_data.transpose(2, 0, 1)])
    segmentation_resized = np.array([resize(slice, IMG_SIZE, mode='constant', preserve_range=True, order=0) for slice in segmentation_data.transpose(2, 0, 1)])

   
    segmentation_onehot = to_categorical(segmentation_resized, num_classes=3)

    return volume_resized, segmentation_onehot


X_train, Y_train = [], []


image_files = sorted([f for f in os.listdir(VOLUME_DIR) if f.endswith(".nii")])
mask_files = sorted([f for f in os.listdir(SEGMENTATION_DIR) if f.endswith(".nii")])

for img_file, mask_file in zip(image_files, mask_files):
    volume_path = os.path.join(VOLUME_DIR, img_file)
    segmentation_path = os.path.join(SEGMENTATION_DIR, mask_file)
    vol, seg = preprocess_data(volume_path, segmentation_path)
    X_train.append(vol)
    Y_train.append(seg)
# Shape: (num_slices, 128, 128)
# Shape: (num_slices, 128, 128, 3)
X_train = np.concatenate(X_train, axis=0)  
Y_train = np.concatenate(Y_train, axis=0)  

print("Training data shape:", X_train.shape)
print("Segmentation mask shape:", Y_train.shape)

In [ ]:
import os
import numpy as np
import nibabel as nib
from tensorflow.keras.utils import to_categorical # type: ignore
# Function to load NIfTI files
def load_nifti(file_path):
    """Load a NIfTI file and return the NumPy array"""
    print(f"Loading: {file_path}")  # Debugging: Print file being loaded
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    nifti_img = nib.load(file_path)
    return nifti_img.get_fdata()

# Function to preprocess test data
def preprocess_data(test_volume_path, test_segmentation_path):
    """Load and preprocess a single test volume and its corresponding segmentation mask"""
    volume_data = load_nifti(test_volume_path)
    segmentation_data = load_nifti(test_segmentation_path).astype(int)

    # Normalize CT images (0-1)
    volume_data = (volume_data - np.min(volume_data)) / (np.max(volume_data) - np.min(volume_data))

    # Resize to (128, 128)
    from skimage.transform import resize
    IMG_SIZE = (128, 128)
    
    volume_resized = np.array([
        resize(slice, IMG_SIZE, mode='constant', preserve_range=True)
        for slice in volume_data.transpose(2, 0, 1)  # Rearrange to (slices, H, W)
    ])
    
    segmentation_resized = np.array([
        resize(slice, IMG_SIZE, mode='constant', preserve_range=True, order=0) 
        for slice in segmentation_data.transpose(2, 0, 1)
    ])

    # One-hot encode segmentation masks (background=0, liver=1, tumor=2)
    
    segmentation_onehot = to_categorical(segmentation_resized, num_classes=3)

    return volume_resized, segmentation_onehot

# ---- Load Test Data ----
test_image_path = os.path.join("test_folder") 
test_mask_path = os.path.join("test_folder", "segmentation") 

# Ensure directories exist
if not os.path.exists(test_image_path) or not os.path.exists(test_mask_path):
    raise FileNotFoundError("Test data folder or segmentation folder not found!")

# Get sorted filenames
image_files = sorted([f for f in os.listdir(test_image_path) if f.endswith(".nii")])
mask_files = sorted([f for f in os.listdir(test_mask_path) if f.endswith(".nii")])

# Check if the number of images and masks match
if len(image_files) != len(mask_files):
    raise ValueError("Mismatch between the number of test images and segmentation masks!")

X_test, Y_test = [], []

# Load and preprocess each test image and mask
for img_file, mask_file in zip(image_files, mask_files):
    volume_path = os.path.join(test_image_path, img_file)  # Correct path joining
    segmentation_path = os.path.join(test_mask_path, mask_file)  # Correct path joining

    vol, seg = preprocess_data(volume_path, segmentation_path)
    X_test.append(vol)
    Y_test.append(seg)

# Convert lists to NumPy arrays
X_test = np.array(X_test)  # Shape: (num_volumes, num_slices, 128, 128)
Y_test = np.array(Y_test)  # Shape: (num_volumes, num_slices, 128, 128, 3)

# Reshape to flatten across all slices
X_test = X_test.reshape(-1, 128, 128, 1)  # Add channel dimension
Y_test = Y_test.reshape(-1, 128, 128, 3)  # Keep segmentation masks in one-hot format

# Print shapes
print("Final Test Data Shape:", X_test.shape)  # (total_slices, 128, 128, 1)
print("Final Test Mask Shape:", Y_test.shape)  # (total_slices, 128, 128, 3)


In [ ]:


# 3. Define the UNet++ model (same as in your original message)
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class UNetPlusPlus(nn.Module):
    def __init__(self, in_channels=1, out_channels=3, filters=[32, 64, 128, 256, 512]):
        super().__init__()
        self.conv0_0 = ConvBlock(in_channels, filters[0])
        self.pool0 = nn.MaxPool2d(2)
        self.conv1_0 = ConvBlock(filters[0], filters[1])
        self.pool1 = nn.MaxPool2d(2)
        self.conv2_0 = ConvBlock(filters[1], filters[2])
        self.pool2 = nn.MaxPool2d(2)
        self.conv3_0 = ConvBlock(filters[2], filters[3])
        self.pool3 = nn.MaxPool2d(2)
        self.conv4_0 = ConvBlock(filters[3], filters[4])

        self.up1_0 = nn.ConvTranspose2d(filters[1], filters[0], kernel_size=2, stride=2)
        self.conv0_1 = ConvBlock(filters[0]*2, filters[0])
        self.up2_0 = nn.ConvTranspose2d(filters[2], filters[1], kernel_size=2, stride=2)
        self.conv1_1 = ConvBlock(filters[1]*2, filters[1])
        self.up3_0 = nn.ConvTranspose2d(filters[3], filters[2], kernel_size=2, stride=2)
        self.conv2_1 = ConvBlock(filters[2]*2, filters[2])
        self.up4_0 = nn.ConvTranspose2d(filters[4], filters[3], kernel_size=2, stride=2)
        self.conv3_1 = ConvBlock(filters[3]*2, filters[3])

        self.up1_1 = nn.ConvTranspose2d(filters[1], filters[0], kernel_size=2, stride=2)
        self.conv0_2 = ConvBlock(filters[0]*3, filters[0])
        self.up2_1 = nn.ConvTranspose2d(filters[2], filters[1], kernel_size=2, stride=2)
        self.conv1_2 = ConvBlock(filters[1]*3, filters[1])
        self.up3_1 = nn.ConvTranspose2d(filters[3], filters[2], kernel_size=2, stride=2)
        self.conv2_2 = ConvBlock(filters[2]*3, filters[2])

        self.final_conv = nn.Conv2d(filters[0], out_channels, kernel_size=1)

    def forward(self, x):
        x0_0 = self.conv0_0(x)
        x1_0 = self.conv1_0(self.pool0(x0_0))
        x2_0 = self.conv2_0(self.pool1(x1_0))
        x3_0 = self.conv3_0(self.pool2(x2_0))
        x4_0 = self.conv4_0(self.pool3(x3_0))

        x3_1 = self.conv3_1(torch.cat([self.up4_0(x4_0), x3_0], 1))
        x2_1 = self.conv2_1(torch.cat([self.up3_0(x3_0), x2_0], 1))
        x1_1 = self.conv1_1(torch.cat([self.up2_0(x2_0), x1_0], 1))
        x0_1 = self.conv0_1(torch.cat([self.up1_0(x1_0), x0_0], 1))

        x2_2 = self.conv2_2(torch.cat([self.up3_1(x3_1), x2_0, x2_1], 1))
        x1_2 = self.conv1_2(torch.cat([self.up2_1(x2_1), x1_0, x1_1], 1))
        x0_2 = self.conv0_2(torch.cat([self.up1_1(x1_1), x0_0, x0_1], 1))

        return self.final_conv(x0_2)



In [ ]:
# Convert test data to tensors
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)  # (N, 1, 128, 128)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32).permute(0, 3, 1, 2)  # (N, 3, 128, 128)

# Create a DataLoader with all slices in one batch
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [ ]:
# 4. Training Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
model = UNetPlusPlus().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 5. Training Loop
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        targets = torch.argmax(targets, dim=1)  # Convert one-hot to class indices

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {epoch_loss / len(train_loader):.4f}")

In [ ]:
def compute_metrics(y_true, y_pred, num_classes=3):
    smooth = 1e-6
    metrics = {}

    for cls in range(num_classes):
        y_true_cls = (y_true == cls)
        y_pred_cls = (y_pred == cls)

        intersection = (y_true_cls & y_pred_cls).sum().item()
        union = (y_true_cls | y_pred_cls).sum().item()
        tp = intersection
        fp = y_pred_cls.sum().item() - tp
        fn = y_true_cls.sum().item() - tp

        precision = tp / (tp + fp + smooth)
        recall = tp / (tp + fn + smooth)
        f1 = 2 * precision * recall / (precision + recall + smooth)
        iou = intersection / (union + smooth)
        dice = 2 * intersection / (y_pred_cls.sum().item() + y_true_cls.sum().item() + smooth)

        metrics[cls] = {
            'IoU': iou,
            'Precision': precision,
            'Recall': recall,
            'F1-score': f1,
            'Dice': dice
        }

    # Dice for class 1 + class 2 (liver + tumor)
    liver = (y_true == 1)
    tumor = (y_true == 2)
    pred_liver = (y_pred == 1)
    pred_tumor = (y_pred == 2)

    combined_true = liver | tumor
    combined_pred = pred_liver | pred_tumor
    intersection = (combined_true & combined_pred).sum().item()
    dice_combined = 2 * intersection / (combined_true.sum().item() + combined_pred.sum().item() + smooth)

    return metrics, dice_combined

In [ ]:
model.eval()
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model(inputs)
        predictions = torch.argmax(outputs, dim=1)  # (N, 128, 128)
        true_classes = torch.argmax(targets, dim=1)  # (N, 128, 128)

metrics, dice_combined = compute_metrics(true_classes.cpu(), predictions.cpu())

# Print all class-wise metrics
for cls in metrics:
    print(f"Metrics for Class {cls}:")
    print(f"IoU: {metrics[cls]['IoU']:.4f}")
    print(f"Precision: {metrics[cls]['Precision']:.4f}")
    print(f"Recall: {metrics[cls]['Recall']:.4f}")
    print(f"F1-score: {metrics[cls]['F1-score']:.4f}")
    print(f"Dice Coefficient: {metrics[cls]['Dice']:.4f}\n")

print(f"Dice Coefficient for Liver + Tumor combined: {dice_combined:.4f}")